# SECTION 1 - Introduction to the problem/task and dataset

PumpkinSeed-ML-Insights is a comprehensive repository dedicated to the exploration and analysis of Turkish pumpkin seed varieties, with a focus on classifying whether a seed belongs to Urgup Sivrisi or Cercevelik species. This project demonstrates the knowledge of authors in data science and machine learning.

Within this repository, you will find a Jupyter Notebook that serves as a self-explanatory document, guiding you through the entire process. This repository also contains three Python files, each implementing a different machine learning model: `knn_pumpkinseed.py`, `logistic_regression_pumkinseed.py`, and `neural_network_pumpkinseed.py`. There is also the `pumpkin_seeds.csv`, which contains the data and `Pumpkin_seeds.pdf`, which contains some description of the dataset.



---
# SECTION 2 - Description of the dataset

`pumpkin_seeds.csv` is a CSV file containing information about Pumpkin Seeds found in Turkey. 

This dataset came from the study `The use of machine learning methods in classification of pumpkin seeds (Cucurbita pepo L.).` by Koklu, M., Sarigil, S., & Ozbek, O. in 2021. In their paper, they used a product shooting box to obtain quality images of the pumpkin seeds. The authors converted the images to a gray tone and then to binary images. To convert the image data into a CSV file, they extracted 12 morphological features.

Overall, the CSV file has 13 columns/features and 2500 rows. The first 12 columns are from the extracted morphological features, while the last column classifies whether it belongs to the Urgup Sivrisi or Cercevelik species. There are 2500 rows, representing a single seed used in the study. There are 1200 Urgup Sivrisi and 1300 Cercevelik species of pumpkin seeds. 

The features found in this CSV file are as follows:
1. Area – Number of pixels within the borders of a pumpkin seed
2. Perimeter – Circumference in pixels of a pumpkin seed
3. Major_Axis_Length – Large axis distance of a pumpkin seed
4. Minor_Axis_Length – Small axis distance of a pumpkin seed
5. Convex_Area – Number of pixels of the smallest convex shell at the region formed by the
pumpkin seed.
6. Equiv_Diameter – Computed as !4𝑎⁄𝜋, where 𝑎 is the area of the pumpkin seed.
7. Eccentricity – Eccentricity of a pumpkin seed
8. Solidity – Convex condition of the pumpkin seeds
9. Extent – Ratio of a pumpkin seed area to the bounding box pixels
10. Roundness – Ovality of pumpkin seeds without considering the distortion of the edges.
11. Aspect_Ration – Aspect ratio of the pumpkin seeds
12. Compactness – Proportion of the area of the pumpkin seed relative to the area of the circle
with the same circumference
13. Class – Either Cercevelik or Urgup Sivrisi



---
# SECTION 3 - List of requirements

---
## SECTION 4 - Data preprocessing and cleaning



The first step is to address the encoding issues, especially in the "Class" column. The unique values in the "Class" column are showing encoding issues, as evidenced by the presence of escape characters like \x82. These values are intended to represent the two species of pumpkin seeds. To correct this, we need to replace these incorrectly encoded strings with a correct format. We replaced the string class names with an integer value of 0 and 1.

In [ ]:
import numpy as np
import pandas as pd
import csv
import seaborn as sns


In [ ]:
data = []

with open('pumpkin_seeds.csv', 'r', encoding='utf-8', errors='replace') as csv_file:
    raw_data = csv.reader(csv_file)

    #Skip headers
    next(raw_data)

    #Store data into data array
    for row in raw_data:
        row_data = []
        for i in range(13): #Convert errors into 1 or 2 (depending on their specie)
            if i == 12 and row[i] == '�er�evelik':
                row_data.append(int(0))
            elif i == 12 and row[i] == '�rg�p Sivrisi':
                row_data.append(int(1))
            else:
                row_data.append(row[i])

        data.append(row_data)

#Convert data into numpy array
np_data = np.array(data)
np_data



## Data Scaling

The numerical features are scaled using `StandardScaler` from `sklearn.preprocessing`. This ensures that all features have a mean of 0 and a standard deviation of 1, which is particularly important for many machine learning algorithms.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Replace 'original_column_names' with the actual list of column names
original_column_names = ['area', 'perimeter', 'major_axis_length', 'minor_axis_length',
                         'convex_area', 'equiv_diameter', 'eccentricity', 'solidity',
                         'extent', 'roundness', 'aspect_Ration', 'compactness', 'class']

# Convert the numpy array to a pandas DataFrame using the original column names
pumpkin_seeds_data = pd.DataFrame(np_data, columns=original_column_names)

# Selecting only the numerical features for scaling
numerical_features = pumpkin_seeds_data.iloc[:, :-1]

# Initializing the Standard Scaler
scaler = StandardScaler()

# Scaling the numerical features
scaled_numerical_features = scaler.fit_transform(numerical_features)

# Creating a new DataFrame with scaled values using the original column names (excluding 'class')
scaled_numerical_df = pd.DataFrame(scaled_numerical_features, columns=original_column_names[:-1])

# Adding the non-numerical column ('class') back to the DataFrame
scaled_pumpkin_seeds_data = pd.concat([scaled_numerical_df, pumpkin_seeds_data['class']], axis=1)

scaled_pumpkin_seeds_data


After cleaning the data, it is now time to define our X and y variables. The X variable will hold the features, while the y variable contains our target. 

In [ ]:
# Creating the feature set 'X' and the target 'y'
X = scaled_pumpkin_seeds_data.iloc[:, :-1]  # All columns except the last one
y = scaled_pumpkin_seeds_data['class']      # Only the last column


Let's separate the training from the test set. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

print('X_train shape', X_train.shape)
print('y_train shape', X_test.shape)
print('X_test shape', y_train.shape)
print('y_test shape', y_test.shape)


---------------------------------------------------------------------------
# SECTION 5 - Exploratory Data Analysis (EDA)

## Descriptive Statistics
Here, we examine the descriptive statistics of the numerical features. This includes measures like mean, median, standard deviation, and quartiles, which provide insights into the central tendency and spread of the data.

In [ ]:
# Descriptive Statistics for Numerical Features
descriptive_stats = scaled_pumpkin_seeds_data.describe()
descriptive_stats


In our exploratory data analysis, the descriptive statistics of the scaled pumpkin seed dataset reveal insightful trends and characteristics. Each feature, represented by 2,500 observations, exhibits a mean value near zero and a standard deviation close to one, a clear indication of effective standard scaling. This normalization is crucial for many machine learning algorithms as it ensures that all features contribute equally to the analysis without bias from differing scales. The minimum and maximum values across the features range significantly, suggesting a wide spread in the data, which is pivotal in understanding the distribution's extent and variance. While these scaled statistics inherently mask the actual raw values, they serve as a vital step in preparing the data for subsequent predictive modeling.

## Feature Distributions

In [ ]:
# Visualizing the Distribution of Each Feature
import matplotlib.pyplot as plt

features = scaled_pumpkin_seeds_data.columns[:-1]  # Excluding the class column
for feature in features:
    plt.figure(figsize=(6, 4))
    plt.hist(scaled_pumpkin_seeds_data[feature], bins=20, alpha=0.7)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.show()


The histogram visualizations of the scaled pumpkin seed dataset provide information into the distributions of each feature. These graphical representations, with their focus on frequency distributions, highlight key characteristics such as skewness, modality, and the spread of data. For features with distributions skewing left or right, we observe an asymmetrical spread, indicating a higher concentration of data points on one side of the mean. This skewness can be crucial in understanding the natural tendencies of certain features. Features exhibiting a more symmetrical distribution around the zero mark suggest a relatively even spread on both sides of the mean, implying no significant skewness. The presence of multiple peaks in some histograms hints at multimodality, which could be a sign of underlying subgroups within the data. Additionally, the spread and range of the data within these histograms, particularly any data points lying far from the central cluster, might signal outliers or unusual variations.

## Box Plots for Numerical Features

In [ ]:
# Box Plots for Each Feature
for feature in features:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=scaled_pumpkin_seeds_data[feature])
    plt.title(f'Box Plot of {feature}')
    plt.show()


The box plots for each feature in our scaled pumpkin seed dataset offer valuable insights into the distribution characteristics of our data. Each plot vividly illustrates the median, quartiles, and potential outliers, providing a comprehensive view of data dispersion and central tendency. Features with wider boxes indicate greater dispersion in values, whereas narrower boxes suggest more uniform data. The whiskers extending from each box plot further reveal the range of the data, excluding outliers. Points that lie beyond these whiskers are marked as outliers, highlighting exceptional values that may warrant further investigation. These outliers could represent anomalies or unique characteristics of the dataset, and their presence is crucial for understanding the full scope of the data's behavior.

---------------------------------------------------------------------------
# SECTION 6 - Model training

## K-Nearest Neighbors (KNN)
The K-Nearest Neighbors (KNN) algorithm is often used for classification task. It clasifies data points based on the majority class of their k-nearest neighbors. While it is a simple algorithm, one of its advantages is that it does not make strong assumptions about the underlying data distribution. This makes it versatile and suitable for various types of datasets, including this dataset.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
# Initializing the KNN classifier
model = KNeighborsClassifier()


In [ ]:
# Training the model on the training data
model.fit(X_train, y_train)


In [ ]:
# Calculating the accuracy of the model on the test data
accuracy = model.score(X_test, y_test)
print('Accuracy:', accuracy*100, '%')


In [ ]:
# Setting arbitrary value (5) for the number of neighbors
neighbors = 5


### Number of Neighbors
This hyperparameter is crucial in KNN which represent the number of neighbors to consider when making predcitions.

The value assigned to the variable neighbors is chosen arbitrarily to assess the model's performance improvement following the cross-validation of hyperparameters.

In [ ]:
distances, data_index = model.kneighbors(X_test,neighbors)


In [ ]:
# Display top neighbors of the test data
np.squeeze(distances)
np.squeeze(data_index)
print('Top 5 neighbors of the test data:')
for i in range(5):
    print('Neighbor:', i+1)
    print('Distance:', distances[i])
    print('Data index:', data_index[i])
    print('Class:', y_train.iloc[data_index[i]])
    print('\n')


## Logistic Regression
The second model we are using is `Logistic Regression`, a binary classifier. Within the context of our dataset, two distinct classes are identified for the pumpkin seeds data, namely Cercevelik and Urgup Sivrisi. The rationale behind selecting logistic regression as a suitable machine learning model for this binary classification problem lies in its capacity to predict the probabilities associated with a given data point belonging to a particular class.

For this model, we will be using the `SGDClassifier`. We are setting the loss parameter to be `log_loss` to use the loss function of a Logistic Regression model. Based from its name, this model uses a stochastic gradient descent as its optimizer. The initial learning rate is `0.001`, which can be altered later after tuning. The learning rate schedule we are using is `constant`, which means we are always using `0.001` to find the minimum of the loss function throughout the training process.

For comparison purposes, we have also incorporated a `mini-batch gradient descent` optimization algorithm to facilitate the training of the logistic regression model. This iterative optimization technique enhances the efficiency of parameter updates by processing a subset, or mini-batch, of the entire dataset at each iteration.


In [ ]:
# Instantiate the Logistic Regression model
from sklearn.linear_model import SGDClassifier

logi_model = SGDClassifier(loss = 'log_loss', eta0 = 0.001, max_iter = 200,
                            learning_rate = 'constant', random_state = 42, verbose = 0)


In [ ]:
# Convert back to numpy array since dataloader (for minibatch gradient descent) expects numpy array
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [ ]:
# Train the model and get the predictions
logi_model.fit(X_train, y_train)
predictions_train = logi_model.predict(X_train)
predictions_test = logi_model.predict(X_test)


In [ ]:
def compute_accuracy(predictions, actual):

    correct = np.sum(predictions == actual)

    accuracy = correct/len(predictions) * 100.0

    return accuracy


In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")
print("Testing accuracy: ", compute_accuracy(y_test, predictions_test),"%")


In [ ]:
# Using minibatch gradient descent instead of stochastic gradient descent
logi_model_bgd = SGDClassifier(loss = 'log_loss', eta0 = 0.001, learning_rate = 'constant', random_state = 42, verbose = 0)


In [ ]:
# Use data_loader python file from our past notebooks
from data_loader import DataLoader
data_loader = DataLoader(X_train, y_train, 10)


In [ ]:
from sklearn.metrics import log_loss

max_epochs = 200
e = 0
is_converged = False
previous_loss = 0
labels = np.unique(y_train)

# For each epoch
while e < max_epochs and is_converged is not True:

    loss = 0

    X_batch, y_batch = data_loader.get_batch()

    for i, j in zip(X_batch, y_batch):

        # Partial fit the model
        logi_model_bgd.partial_fit(i,j,labels)

        # Compute the loss
        y_pred = logi_model_bgd.predict_proba(X_train)
        loss += log_loss(y_train, y_pred)

    # Display the average loss per epoch
    print('Epoch:', e + 1, '\tLoss:', (loss / len(X_batch)))

    if abs(previous_loss - loss) < 0.005:
        is_converged = True
    else:
        previous_loss = loss
        e += 1


In [ ]:
predictions_train = logi_model_bgd.predict(X_train)
predictions_test = logi_model_bgd.predict(X_test)


In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")
print("Testing accuracy: ", compute_accuracy(y_test, predictions_test),"%")


## Naive Bayes

The third model we will be evaluating is the Gaussian Naive Bayes model. This model is based on the Bayes theorem, which describes the probability of an event based on prior knowledge of conditions that might be related to the event. The Gaussian Naive Bayes model assumes that the features are normally distributed, which is generally true for our dataset as seen in our EDA. We chose the Gaussian variant over the multinomial variant because our dataset is continuous and not discrete. The multinomial variant is used for discrete features, such as word counts for text classification.

In [ ]:
from sklearn.naive_bayes import GaussianNB
seed_nb = GaussianNB()
seed_nb.fit(X_train, y_train)
print(seed_nb.class_count_)
print("Prior: ", seed_nb.class_prior_)


The GNB model is implemented using the `GaussianNB` class from `sklearn.naive_bayes`. `GaussianNB()` has two parameters: `priors` and `var_smoothing`. `priors` is an array of prior probabilities of the classes. If it is not specified, the prior probabilities are automatically adjusted according to the data. `var_smoothing` is a smoothing parameter that is used to compute the variance of the features. The default value is `1e-9`.

In [ ]:
predictions = seed_nb.predict(X_train)

def compute_accuracy(predictions, actual):
    return (np.sum(predictions == actual)/len(actual)) * 100

print("Training accuracy: ", compute_accuracy(predictions, y_train), "%")

predictions = seed_nb.predict(X_test)
predictions
print("Test accuracy: ", compute_accuracy(predictions, y_test), "%")


As you can see, the GNB model has a training accuracy of 87.36% and a test accuracy of 86.08%, which are relatively close to each other. This is a sign that the model is not overfitting nor underfitting.

---
# SECTION 7 - Hyperparameter tuning

## K-Nearest Neighbors (kNN)

These will be the values used for cross-validation. Cross-validation is used to assess the model's performance across different folds.

Setting `k_folds = 5` means that the dataset will be divided into 5 roughly equal-sized subsets. This is to be able to assess the performance of the model more reliably. The given value is a good balance between reducing variability in the estimate and keeping computational cost reasonable.

Unlike other algorithms, KNN does not involve a learning process like other iterative algorithms. This model directly memorizes the training data.

In [ ]:
# Import cross validation libraries
from sklearn.model_selection import cross_val_score


In [ ]:
k_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]
scores = np.zeros((len(k_choices), k_folds))


In [ ]:
for i in range(len(k_choices)):
    model = KNeighborsClassifier(n_neighbors=k_choices[i])
    scores[i] = cross_val_score(model, X_train, y_train, cv=k_folds)


In [ ]:
def plot_scatter(scores):
    for i in range(len(scores)):
        x=[k_choices[i]] * 5
        plt.scatter(x, scores[i])

plot_scatter(scores)


In [ ]:
# Calculate the average scores of each fold
avg_scores = np.mean(scores, axis=1)
print('Average scores:', avg_scores)


In [ ]:
# Get average accuracy for each k
stddev_scores = np.std(scores, axis=1)
print('Standard deviation of scores:', stddev_scores)


In [ ]:
plot_scatter(scores)

plt.errorbar(k_choices, avg_scores, yerr=stddev_scores)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')


In [ ]:
# Test on selected k value
model = KNeighborsClassifier(n_neighbors=15)

# Training the model on the training data
model.fit(X_train, y_train)


In [ ]:
# Calculating the accuracy of the model on the test data
y_pred = model.predict(X_test)
accuracy = model.score(X_test, y_test)
print('Accuracy:', accuracy*100, '%')


The accuracy of the model demonstrates improvement as the number of neighbors increases, peaking at 15. This particular value of `n_neighbors` yields the highest average accuracy at 87.2%, indicating a balanced trade-off between bias and variance.

## Logistic Regression

To fine tune our model, we plan to have these hyperparameter options be assessed by `RandomizedSearchCV`.

These hyperparameters are relevant in our `SGDClassifier` using `Logistic Regression`.

1. `alpha` is the regularization parameter strength. This helps in preventing overfitting the model to the training data. The greater the alpha, the stronger the regularization. However, there is a possibility that if alpha is too big, it can lead to underfitting.

2. `l1_ratio` controls the balance between `l1` and `l2` regularization. A value of 0 means it's using `l2` regularization and a value of 1 means it's using `l1` regularization. 

3. `tol` is the tolerance for the stopping criterion. It serves as the threshold at which the algorithm will stop iterating.

4. `eta0` is the initial learning rate. It controls the step size at each iteration. Since we're using `constant` as our learning_rate schedule, the value of `eta0` never changes.

5. `penalty` is the regularization term to be used. 

In [ ]:
hyperparameters = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'l1_ratio': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'tol': [0.0001, 0.001, 0.01, 0.1],
    'eta0': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2', 'l1', 'elasticnet']
}


In [ ]:
logi_model_hyper = SGDClassifier(loss='log_loss', learning_rate='constant', random_state=42, verbose=0)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_search_logi_model = RandomizedSearchCV(estimator=logi_model_hyper, param_distributions=hyperparameters, n_iter=100, cv=5, random_state=42)


In [ ]:
random_search_logi_model.fit(X_train, y_train)


In [ ]:
print(random_search_logi_model.best_estimator_)


In [ ]:
# Instantiate SGDClassifier using the best estimators found by RandomizedSearchCV
logi_model_besthyper = SGDClassifier(eta0=0.001, l1_ratio=0, learning_rate='constant', loss='log_loss', penalty='elasticnet', random_state=42, tol=0.0001)


In [ ]:
# Train the model and get the predictions
logi_model_besthyper.fit(X_train, y_train)
predictions_train = logi_model_besthyper.predict(X_train)
predictions_test = logi_model_besthyper.predict(X_test)


In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")
print("Testing accuracy: ", compute_accuracy(y_test, predictions_test),"%")


The performance of our model without hyperparameter tuning is as follows:  
Training accuracy:  `88.16000000000001 %`  
Testing accuracy:  `85.11999999999999 %`

After tuning, we got:  
Training accuracy:  `88.32 %`  
Testing accuracy:  `85.44 %`

Hyperparameter tuning has shown an improvement in our accuracies.

In [ ]:
# Using minibatch gradient descent instead of stochastic gradient descent
logi_model_besthyper_bgd = SGDClassifier(eta0=0.001, l1_ratio=0, learning_rate='constant', loss='log_loss', penalty='elasticnet', random_state=42, tol=0.0001)


In [ ]:
from sklearn.metrics import log_loss

max_epochs = 200
e = 0
is_converged = False
previous_loss = 0
labels = np.unique(y_train)

# For each epoch
while e < max_epochs and is_converged is not True:

    loss = 0

    X_batch, y_batch = data_loader.get_batch()

    for i, j in zip(X_batch, y_batch):

        # Partial fit the model
        logi_model_besthyper_bgd.partial_fit(i,j,labels)

        # Compute the loss
        y_pred = logi_model_besthyper_bgd.predict_proba(X_train)
        loss += log_loss(y_train, y_pred)

    # Display the average loss per epoch
    print('Epoch:', e + 1, '\tLoss:', (loss / len(X_batch)))

    if abs(previous_loss - loss) < 0.005:
        is_converged = True
    else:
        previous_loss = loss
        e += 1


In [ ]:
predictions_train = logi_model_besthyper_bgd.predict(X_train)
predictions_test = logi_model_besthyper_bgd.predict(X_test)


In [ ]:
print("Training accuracy: ", compute_accuracy(y_train, predictions_train),"%")
print("Testing accuracy: ", compute_accuracy(y_test, predictions_test),"%")


The performance of our model in mini-batch gradient descent without hyperparameter tuning is as follows:  
Training accuracy:  `88.32 %`  
Testing accuracy:  `85.92 %`

After tuning, we got:  
Training accuracy:  `88.26666666666667 %`  
Testing accuracy:  `86.24000000000001 %`

Unfortunately, in mini-batch gradient descent, the hyperparameter tuning did worse. A possible reason for this is the RandomizedSearchCV class only accounted for stochastic gradient descent and not for batch gradient descent. Another reason is inside the algorithm for mini-batch gradient descent, we've been fitting the model to different data, hence getting different accuracies.

The batch size used in the data loader is 10. A higher batch size resulted to the model performing worse. Lowering the batch size made the model perform better, but made it slower. 

The accuracies when the batch size used in the data loader is 5 are as follows:

Without hypertuning  
Training accuracy:  `88.37333333333333 %`  
Testing accuracy:  `86.08 %`

With hypertuning  
Training accuracy:  `88.26666666666667 % `  
Testing accuracy:  `86.08  %`

As you can see, the performance is similar. The only issue is the speed. Therefore, we set the batch_size to 10.

## Naive Bayes

In order to tune the hyperparameters of the GNB model, we used the `ParameterGrid` class from `sklearn.model_selection`. `ParameterGrid` is an iterable that iterates over the hyperparameter space. For this case, the hyperparameter we will be tuning is `var_smoothing`. The reason we will not be tuning `priors` is because leaving it unspecified will let the model automatically adjust the prior probabilities according to the data, which is what we would be doing if we were tuning it anyways.

We will first split the dataset into training, validation, and test sets.

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("X_test shape : ", X_test.shape)
print("y_test shape : ", y_test.shape)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

print("X_train shape : ", X_train.shape)
print("y_train shape : ", y_train.shape)
print("X_validation shape : ", X_validation.shape)
print("y_validation shape : ", y_validation.shape)


Then, we will define the range of values to use for `var_smoothing`. We will use a logarithmic scale for the range of values, as the values for `var_smoothing` are very small. We will use the `np.logspace` function from `numpy` to generate the range of values.

In [ ]:
from sklearn.model_selection import ParameterGrid
seed_nb = GaussianNB()
hyperparameters = [{
    'var_smoothing' : np.logspace(-10, -5, num=100),
}]


Then, for every value of `var_smoothing` in the range, we will create a GNB model with that value and train it using the training set. Then, we will evaluate the model using the validation set. The hyperparameter value that gives the best validation accuracy will be assigned to `best_grid`.

In [ ]:
best_score = 0
for g in ParameterGrid(hyperparameters):
    print(g)

    seed_nb.set_params(**g)

    seed_nb.fit(X_train, y_train)
    predictions = seed_nb.predict(X_train)
    train_acc = compute_accuracy(predictions, y_train)

    predictions = seed_nb.predict(X_validation)
    val_acc = compute_accuracy(predictions, y_validation)

    print(f"Train acc: {train_acc}% \t Val acc: {val_acc}%", end="\n\n")

    if val_acc > best_score:
        best_score = val_acc
        best_grid = g

print("Best accuracy: ", best_score, "%")
print("Best grid: ", best_grid)


As you can see, the validation and training accuracy during hyperparameter tuning do not vary at all. This could be a sign that the variances of the features do not vary much. Thus, `var_smoothing` may not be very influential for this dataset.

Now, we'll create a GNB model using the best hyperparameter values, which in this case is `1e-10`. Do note that it chose `1e-10` because it was the first parameter value tested, but because the `var_smoothing` parameter doesn't seem to affect the accuracy at all, it may as well just be set to the default value.

In [ ]:
seed_nb = GaussianNB(var_smoothing=1e-10)
print(seed_nb.get_params())
seed_nb.fit(X_train_val, y_train_val)

predictions = seed_nb.predict(X_test)
test_acc = compute_accuracy(predictions, y_test)
print("Test accuracy: ", test_acc, "%")


We see that the test accuracy for this test set is at 85.8%, which is slightly lower than the test accuracy of the model without hyperparameter tuning. However, we can try to use the same training and testing sets used earlier with the new parameter to see how it fares with the same test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [ ]:
seed_nb.fit(X_train, y_train)
predictions = seed_nb.predict(X_test)

test_acc = compute_accuracy(predictions, y_test)
print("Test accuracy: ", test_acc, "%")


We can see that the test accuracy is the same as last time, 86.08%. This makes sense, as during the hyperparameter tuning, it was shown that adjusting the `var_smoothing` parameter did not influence the accuracy of the model at all. Therefore, the model with the default hyperparameter values is the best model for this dataset.

##

---

# SECTION 8 - Model selection

**kNN**: 

``KNeighborsClassifier(n_neighbors=15)``

Accuracy = 87.20%

**Logistic Regression**

``SGDClassifier(eta0=0.001, l1_ratio=0, learning_rate='constant', loss='log_loss', penalty='elasticnet', random_state=42, tol=0.0001)``

Accuracy = 85.44%

**Gaussian Naive Bayes**

``GaussianNB(var_smoothing=1e-10)``

Accuracy = 86.08%


Out of the three models, kNN performed the best by a small margin. This may be because kNN does not make strong assumptions about the underlying data distribution, which makes it versatile and suitable for various types of datasets, including this dataset. 

However, that is not to say that the other models performed poorly. Logistic Regression and Naive Bayes performed similarly to kNN. But they may not have performed any better than kNN because they both rely on a few assumptions about the dataset, such as features being independent, which we saw in the EDA that some features in the dataset do have correlations with one another and the data being normally distributed, which some of the features have slightly skewed distributions.

However if we are basing our decision on the accuracy alone, kNN is the best model for this dataset.

---
# SECTION 9 - Insights and conclusion